In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#https://scipython.com/blog/quadtrees-2-implementation-in-python/
#https://pydoc.net/openslide-python/1.1.1/openslide/

import numpy as np
import openslide
from probreg import cpd
from probreg import transformation as tf
import cv2
from PIL import Image
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json

import pickle

In [3]:
## HACK get rid of it
import sys
sys.path.append("..")
from registration_tree import Rect, QuadTree

In [4]:
source_slide_paths = list(Path(r"\\born_pix.fs-de.local\born_pix\CM\ColonCancer\exact_1_11").glob("*.tif"))
target_slide_paths  = list(Path(r"\\born_pix.fs-de.local\born_pix\CM\ColonCancer\exact_1_12").glob("*.tif"))

if len(source_slide_paths) == 0:
    source_slide_paths = list(Path(r"/data/ScannerStudy/MultiSlide/IHC").glob("*.tif"))

if len(target_slide_paths) == 0:
    target_slide_paths = list(Path(r"/data/ScannerStudy/MultiSlide/HE").glob("*.tif"))

In [8]:
source_slide_dict = {name.stem: #.replace(" HE", ""): 
                      {"name": name, "slide": openslide.OpenSlide(str(name))}
                      for name in source_slide_paths}
source_slide_dict.keys()

dict_keys(['01_a', '02_a', '03_a', '03_c', '03_d', '03_e', '03_f', '04_a', '06_a', '06_b', '06_d', '07_a', '07_b', '07_c', '07_d', '07_e', '07_f', '07_g', '09_b', '10_a', '11_a', '11_c', '11_e', '12_a', '13_a'])

In [9]:
target_slide_dict = {name.stem: #.replace(" HE", ""):  
                      {"name": name, "slide": openslide.OpenSlide(str(name))}
                      for name in target_slide_paths}
target_slide_dict.keys()

dict_keys(['01_a', '02_a', '04_a', '06_a', '06_b', '07_a', '07_b', '07_c', '07_d', '07_e', '07_f', '09_b', '10_a', '11_a', '11_c', '11_e', '12_a', '13_a'])

In [10]:
for name in source_slide_dict:
    if name not in target_slide_dict:
        print(name)

03_a
03_c
03_d
03_e
03_f
06_d
07_g


In [11]:
for name in target_slide_dict:
    if name not in target_slide_dict:
        print(name)

In [12]:
%%time

results = {}
#target_depth = 2

for thumbnail_size in [(2048, 2048)]: #(1024, 1024), (2048, 2048), (4096, 4096)
    
    for target_depth in [0]: # , 1, 2
        
        result_folder = Path(f"/MultiSlide/Depth_{target_depth}/{thumbnail_size[0]}")

        result_folder.mkdir(parents=True, exist_ok=True)
    
        for name, source in source_slide_dict.items():
            
            if name not in target_slide_dict:
                continue

            parameter = {
                # feature extractor parameters
                "point_extractor": "sift",  #orb , sift
                "maxFeatures": 2048, 
                "crossCheck": False, 
                "flann": False,
                "ratio": 0.7, 
                "use_gray": True,

                # QTree parameter 
                "homography": True,
                "filter_outliner": False,
                "debug": True,
                "target_depth": target_depth,
                "run_async": False,
                "thumbnail_size": thumbnail_size
            }

            target = target_slide_dict[name]


            source_slide = source["slide"]
            target_slide = target["slide"]

            source_dimension = Rect.create(Rect, 0, 0, source_slide.dimensions[0], source_slide.dimensions[1])
            target_dimension = Rect.create(Rect, 0, 0, target_slide.dimensions[0], target_slide.dimensions[1])

            qtree = QuadTree(source_dimension, source_slide, target_dimension, target_slide, **parameter)

            results[source["name"]] = qtree
            
            with open(result_folder / f'{qtree.source_name}-To-{qtree.target_name}.pickle', 'wb') as handle:
                pickle.dump(qtree, handle, protocol=pickle.HIGHEST_PROTOCOL)

Wall time: 4min 42s


In [15]:

for idx, (path, qtree) in enumerate(results.items()):
    
    #if qtree.target_depth != 2:
    #    continue
    
    #qtree.draw_feature_points(num_sub_pic=5, figsize=(12, 12))
    
    #if idx == 5:
    #    break


SyntaxError: unexpected EOF while parsing (<ipython-input-15-07cab2da6681>, line 9)

In [14]:
for path, qtree in results.items():
        
    print(qtree)

Source: 01_a 
Target: 01_a 
Source: (0.00, 0.00, 44768.00, 48000.00) Target: (0.00, 0.00, 44704.00, 48192.00)
x: [1.000, -0.002, 405.333], y: [0.002, 1.004, 405.007]] error: 260.685
Source: 02_a 
Target: 02_a 
Source: (0.00, 0.00, 47856.00, 54512.00) Target: (0.00, 0.00, 51504.00, 57040.00)
x: [1.013, -0.005, 936.412], y: [0.003, 0.998, 790.671]] error: 888.716
Source: 04_a 
Target: 04_a 
Source: (0.00, 0.00, 44704.00, 64352.00) Target: (0.00, 0.00, 44640.00, 64192.00)
x: [0.997, -0.001, -118.636], y: [0.008, 1.004, 397.273]] error: 1074.955
Source: 06_a 
Target: 06_a 
Source: (0.00, 0.00, 54624.00, 70656.00) Target: (0.00, 0.00, 56528.00, 63440.00)
x: [0.999, 0.003, -1784.845], y: [-0.003, 1.005, 481.270]] error: 1402.371
Source: 06_b 
Target: 06_b 
Source: (0.00, 0.00, 53072.00, 62288.00) Target: (0.00, 0.00, 54784.00, 60784.00)
x: [0.999, 0.001, -3676.217], y: [-0.000, 1.006, -2131.713]] error: 787.109
Source: 07_a 
Target: 07_a 
Source: (0.00, 0.00, 43136.00, 68128.00) Target: (0.0